In [1]:
# Attia et. al. model with 6-lead ECG Data
# 6 leads are reduced to 2 independent leads

# ~~~~~~~~~~~~~~~ IMPORTS ~~~~~~~~~~~~~~~
from collections import Counter
import numpy as np
import keras
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from keras.models import Sequential,Model
from keras.layers import concatenate,Activation,Dropout,Dense,ZeroPadding2D
from keras.layers import Input,add,Conv2D, MaxPooling2D,Flatten,BatchNormalization,LSTM
from keras.optimizers import Adam
import os
import zipfile
from pandas import *
from sklearn.model_selection import train_test_split
from CNN_6lead_model_keras import ResNet18
from pandas import *
import json
import os
from keras.utils import to_categorical
import argparse
import time
import random
# import numpy_indexed as npi
from numpy.random import default_rng

from IPython import display

from PIL import Image, ImageOps
import os, os.path
from tqdm import tqdm_notebook as tqdm
import shap

# ~~~~~~~~~~~~~~~ CONNECT TO GPU ~~~~~~~~~~~~~~~

os.environ["OMP_NUM_THREADS"] = "1"

if os.environ.get("CUDA_VISIBLE_DEVICES") is None:
    #Choose GPU 0 as a default if not specified (can set this in Python script that calls this)
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"]="1"

# ~~~~~~~~~~~~~~~ DATA PREPROCESS ~~~~~~~~~~~~~~~
def preprocess(X, y_labels):
  # Change X dims for model compatibility
  # expected shape=(None, 5000, 2, 1)
  X = np.swapaxes(X,1,2)
  X = np.expand_dims(X, axis=3)

  # Change y dims to one-hot encoding, should be (300, 2)
  y = to_categorical(y_labels, dtype ="uint8")

  X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)
  X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

  N_Val = y_valid.shape[0]
  N_Train = y_train.shape[0]

  print ('Training on : ' + str(N_Train) + ' and validating on : ' +str(N_Val))

  n_classes = 2
  return X_train, X_rem, y_train, y_rem, X_valid, X_test, y_valid, y_test, n_classes

# ~~~~~~~~~~~~~~~ DATA FETCH ~~~~~~~~~~~~~~~

def combine_sets(afib_path, control_path, size=20000):
  np.random.seed(777) # Lucky number 7. 
  
  # Of these, select a random sample:
  afib_ecgs =random.choices(os.listdir(afib_path), k=int(size)) # has to be hardcoded in for some reason??!?!?!

  # Set up y values. These are all afib patients.
  y_labels_afib = np.ones(int(size)) # an array of all 1's

  # Now, let's get the healthy patients:
  control_ecgs = random.choices(os.listdir(control_path), k=int(size))
  y_labels_control = np.zeros(int(size))

  # Concatenate the 2 arrays:
  image_filenames = np.concatenate([afib_ecgs, control_ecgs])
  y_labels = np.concatenate([y_labels_afib, y_labels_control]) #.to(device)

  # Now randomize and make sure correct labels line up with ecgs:
  random_indices = np.random.randint(low=0,high=int(size)*2-1,size = (int(size)*2,))
  image_filenames = image_filenames[random_indices]
  y_labels = y_labels[random_indices]

  # Let's verify:
  print(image_filenames[4])
  print(y_labels[4])
  print(image_filenames[4] in afib_ecgs) # if y-label is 1, then this should be true. if y-label is 0, should be false.

  return image_filenames, y_labels


def fetch_data(datatype, size=20000, load=True):

  if datatype == 'spectrogram':
    if load:
      return preprocess(np.load("spectro_X.npy")[0:3000], np.load("spectro_Y.npy")[0:3000])
    afib_path =  '/local1/CSE_XAI/CSE482-XAI/image_data_processing/control_ecgs_as_spectro/'
    control_path = '/local1/CSE_XAI/CSE482-XAI/image_data_processing/afib_ecgs_as_spectro/'
  else: 
    afib_path =  '../../../../../../../local1/CSE_XAI/CSE482-XAI/image_data_processing/control_ecgs_as_plots/'
    control_path = '../../../../../../../local1/CSE_XAI/CSE482-XAI/image_data_processing/afib_ecgs_as_plots/'


  image_filenames, Y = combine_sets(afib_path, control_path, size)


  X = np.zeros((int(size*2), 300, 300))
  counter = 0
  for file in image_filenames:
    
    if counter %1000 == 0:
      print("Retrieved files: ", counter)

    try:
      image = Image.open(afib_path + file)
    except:
      image = Image.open(control_path + file)

    image = ImageOps.grayscale(image)

    if datatype == 'spectrogram':

      im = Image.open("/local1/CSE_XAI/CSE482-XAI/image_data_processing/afib_ecgs_as_spectro/"+file)
      im = ImageOps.grayscale(im)
      im = im.crop((140, 140, 1800, 1400))
      im = im.resize((300, 300), Image.ANTIALIAS)
      
      patient_X = np.asarray(im)

    
    X[counter] = patient_X
    counter += 1
  
  print("saving data into numpy arrays...")
  """
  if datatype == 'spectrogram':
    np.save("spectro_X.npy", X)
    np.save("spectro_Y.npy", Y)
  else:
    np.save("plot_X.npy", X)
    np.save("plot_Y.npy", Y)

    """
  return preprocess(X, Y)

# ~~~~~~~~~~~~~~~ TRAIN MODEL ~~~~~~~~~~~~~~~
def train_model(X_train, X_rem, y_train, y_rem, X_valid, X_test, y_valid, y_test, n_classes, model_name):
  
  model = ResNet18(2)

  opt0 = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.5)
  model.compile(loss='categorical_crossentropy',  optimizer=opt0,  metrics=['accuracy'])

  earlyStopCallback = EarlyStopping(monitor='val_loss', min_delta=0, patience=9,  mode='auto')
  saveBestCallback = ModelCheckpoint(model_name+'weights_only_checkpoint.h5',monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
  reduceLR =ReduceLROnPlateau(monitor = 'val_loss',factor = 0.5,patience = 3,verbose=1,min_lr = 0.00001)
  history = model.fit(X_train, y_train,validation_data=(X_valid, y_valid),epochs=5, batch_size=128, verbose=1, 
                      callbacks=[saveBestCallback,earlyStopCallback,reduceLR]) #class_weight=class_weight

  
  return model

# ~~~~~~~~~~~~~~~ CALCULATE TEST ACCURACY ~~~~~~~~~~~~~~~
def get_test_acc(model, X_test, y_test):
  score = model.evaluate(X_test, y_test, verbose = 0) 

  print('Test loss:', score[0]) 
  print('Test accuracy:', score[1])
  print()
  print("For reference, Attia model reports 83.3% test accuracy.")
  return score[1]

# ~~~~~~~~~~~~~~~ SAVE MODEL ~~~~~~~~~~~~~~~
def save_model(model, name):
  model.save(name)
  print("Saved model to ", name)

def load_model(filename, input_size):
  model = ResNet18(2)
  model.build(input_shape=input_size)
  model.load_weights(filename)
  return model


2023-06-05 10:51:16.255953: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
start_time = time.time()
#parser = argparse.ArgumentParser()
#parser.add_argument("-f", "--full", help="If you would like to use the full dataset (not recommended) use --full, otherwise, sample is used.", action="store_true")
#parser.add_argument("-s", "--spectrogram", help="Use this tag to use spectrogram images instead of plots.", action="store_true")
#parser.add_argument("-l", "--load_data",help="Use this load the data from the npy files.", action="store_true")
#args = parser.parse_args()
size = 3000
datatype = "plot"
#if args.full:
#  size = 'full'
#if args.spectrogram:
datatype = "spectrogram"

model_name = 'CNN_6lead_'+str(size)+'_dataset_'
X_train, X_rem, y_train, y_rem, X_valid, X_test, y_valid, y_test, n_classes = fetch_data(datatype, size, False)

_5209.png
1.0
True
Retrieved files:  0


ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.


Retrieved files:  1000
Retrieved files:  2000
Retrieved files:  3000
Retrieved files:  4000
Retrieved files:  5000
saving data into numpy arrays...
Training on : 4800 and validating on : 600


In [3]:
# model = train_model(X_train, X_rem, y_train, y_rem, X_valid, X_test, y_valid, y_test, n_classes, model_name)

#test_acc = get_test_acc(model, X_test, y_test)
#model_name += '_'+str(test_acc)[2:]

modelfile = modelfile = '/local1/CSE_XAI/CSE482-XAI/CNN/CNN_6lead_3000_dataset_weights_only_checkpoint.h5'
input_size = (10, 224, 224, 1)
model = load_model(modelfile, input_size)
#model = load_model(modelfile, input_size)

#save_model(model, model_name)
end_time = time.time()
print("Elapsed time: ", str(end_time-start_time))

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2023-06-05 10:58:40.960079: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 10:58:40.960744: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 12. Tune using inter_op_parallelism_threads for best performance.


Elapsed time:  438.0730617046356


In [4]:
X_train.shape

(4800, 300, 300, 1)

In [5]:
# class label list
class_names = ['normal', 'afib']
# example image for each class
images_dict = dict()

images_dict[class_names[0]] = X_train[0]
images_dict[class_names[1]] = X_train[4]

images_dict = dict(sorted(images_dict.items()))
print(images_dict)
    
# example image for each class for test set
x_test_dict = dict()

x_test_dict[class_names[0]] = X_test[0]
x_test_dict[class_names[1]] = X_test[2]

# order by class
x_test_each_class = [x_test_dict[i] for i in sorted(x_test_dict)]
x_test_each_class = np.asarray(x_test_each_class)
# Compute predictions

predictions = model(x_test_each_class.reshape((2, 300, 300, 1)))
predicted_class = np.argmax(predictions, axis=1)

{'afib': array([[[255.],
        [255.],
        [255.],
        ...,
        [255.],
        [255.],
        [255.]],

       [[255.],
        [255.],
        [255.],
        ...,
        [255.],
        [255.],
        [255.]],

       [[255.],
        [255.],
        [253.],
        ...,
        [255.],
        [255.],
        [255.]],

       ...,

       [[255.],
        [255.],
        [255.],
        ...,
        [255.],
        [255.],
        [255.]],

       [[255.],
        [255.],
        [255.],
        ...,
        [255.],
        [255.],
        [255.]],

       [[255.],
        [255.],
        [255.],
        ...,
        [255.],
        [255.],
        [255.]]]), 'normal': array([[[255.],
        [255.],
        [255.],
        ...,
        [255.],
        [255.],
        [255.]],

       [[255.],
        [255.],
        [255.],
        ...,
        [255.],
        [255.],
        [255.]],

       [[255.],
        [255.],
        [253.],
        ...,
        [255.],
  

In [6]:
import matplotlib.pyplot as plt

In [7]:
# plot actual and predicted class
def plot_actual_predicted(images, pred_classes):
  fig, axes = plt.subplots(1, 11, figsize=(16, 15))
  axes = axes.flatten()
  
  # plot
  ax = axes[0]
  dummy_array = np.array([[[0, 0, 0, 0]]], dtype='uint8')
  ax.set_title("Base reference")
  ax.set_axis_off()
  ax.imshow(dummy_array, interpolation='nearest')
  # plot image
  for k,v in images.items():
    ax = axes[k+1]
    ax.imshow(v, cmap=plt.cm.binary)
    ax.set_title(f"True: %s \nPredict: %s" % (class_names[k], class_names[pred_classes[k]]))
    ax.set_axis_off()
  plt.tight_layout()
  plt.show()

In [8]:
model.summary()

Model: "res_net18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  3200      
                                                                 
 batch_normalization (BatchN  multiple                 256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 resnet_block (ResnetBlock)  multiple                  74368     
                                                                 
 resnet_block_1 (ResnetBlock  multiple                 74368     
 )                                                               
                                                         

In [9]:
newInput = Input(batch_shape=(1,128,128,1))
newOutputs = model(newInput)
newModel = Model(newInput, newOutputs)
newModel.set_weights(model.get_weights())

background = X_test[:500]
test_images = X_test[500:504]

e = shap.DeepExplainer(newModel, background)
shap_values = e.shap_values(test_images)

shap_numpy = [np.swapaxes(np.swapaxes(s, 1, -1), 1, 2) for s in shap_values]
test_numpy = np.swapaxes(np.swapaxes(test_images.numpy(), 1, -1), 1, 2)

# plot the feature attributions
shap.image_plot(shap_numpy, -test_numpy)

keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


ValueError: Input 0 of layer "model" is incompatible with the layer: expected shape=(None, 128, 128, 1), found shape=(500, 300, 300, 1)